In [ ]:
import json
import os
import requests
import random
import string
import secrets
import time
import re
import collections
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy
import numpy as np
from collections import defaultdict


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random, string, copy
from collections import Counter

class HangmanDataset(Dataset):
    def __init__(self, words, max_word_length=45, reveal_ratio=0.5):
        self.words = [word.lower() for word in words if len(word) <= max_word_length]
        self.max_length = max_word_length
        self.reveal_ratio = reveal_ratio
        self.char_to_idx = {char: i+1 for i, char in enumerate(string.ascii_lowercase)}
        self.char_to_idx['_'] = 0  # blank
        self.char_to_idx['PAD'] = 27

    def __len__(self): return len(self.words) * 80

    def __getitem__(self, idx):
        word = self.words[idx % len(self.words)]
        reveal_count = int(len(word) * self.reveal_ratio)
        revealed = random.sample(range(len(word)), reveal_count) if reveal_count > 0 else []

        word_state = [0] * self.max_length
        for pos in revealed: word_state[pos] = self.char_to_idx[word[pos]]

        target_pos, target_chars, position_context, vowels = [], [], [0]*self.max_length, set('aeiou')
        for i in range(len(word)):
            if i not in revealed:
                ctx = 0
                if i > 0 and word_state[i-1] != 0: ctx += 1
                if i < len(word)-1 and word_state[i+1] != 0: ctx += 2
                if ctx:
                    target_pos.append(i)
                    target_chars.append(self.char_to_idx[word[i]])
                    position_context[i] = ctx

        count_blanks = word_state[:len(word)].count(0)
        blank_vowel_next = [0]*self.max_length
        for i in range(len(word)):
            if word_state[i] == 0:
                l = word[i-1] if i > 0 else 'x'
                r = word[i+1] if i < len(word)-1 else 'x'
                if l in vowels or r in vowels:
                    blank_vowel_next[i] = 1

        max_targets = 10
        while len(target_pos) < max_targets:
            target_pos.append(-1)
            target_chars.append(0)

        return {
            'word_state': torch.tensor(word_state, dtype=torch.long),
            'position_context': torch.tensor(position_context, dtype=torch.long),
            'target_positions': torch.tensor(target_pos[:max_targets], dtype=torch.long),
            'target_chars': torch.tensor(target_chars[:max_targets], dtype=torch.long),
            'word_length': torch.tensor(len(word), dtype=torch.long),
            'blank_count': torch.tensor(count_blanks, dtype=torch.long),
            'next_to_vowel': torch.tensor(blank_vowel_next, dtype=torch.float)
        }


class EnhancedHangmanModel(nn.Module):
    def __init__(self, vocab_size=28, max_len=45, emb_dim=128, hidden_dim=1024, ablate={}):
        super().__init__()
        self.ablate = ablate
        self.char_emb = nn.Embedding(vocab_size, emb_dim)
        self.ctx_emb = nn.Embedding(4, 32)

        self.pattern_cnn = nn.Sequential(
            nn.Conv1d(emb_dim, 64, 3, padding=1), nn.ReLU(), nn.Dropout(0.2),
            nn.Conv1d(64, 64, 3, padding=1), nn.ReLU(), nn.Dropout(0.2)
        )

        self.encoder = nn.LSTM(emb_dim + 32, hidden_dim, bidirectional=True, batch_first=True)

        self.pos_prior_mlp = nn.Sequential(
            nn.Linear(1 + 1 + 64, 32), nn.ReLU(), nn.Dropout(0.2), nn.Linear(32, 26)
        )

        def decoder():
            return nn.Sequential(
                nn.Linear(hidden_dim*2 + 26, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3),

                nn.Linear(hidden_dim, hidden_dim // 2),
                nn.ReLU(),
                nn.Dropout(0.3),

                nn.Linear(hidden_dim // 2, 26)
            )

        self.left_decoder = decoder()
        self.right_decoder = decoder()
        self.both_decoder = decoder()

    def forward(self, word_state, position_context, word_length, blank_count, next_to_vowel):
        B, L = word_state.size()
        emb = self.char_emb(word_state)
        cnn_feat = self.pattern_cnn(emb.transpose(1, 2)).transpose(1, 2)
        ctx = self.ctx_emb(position_context)
        encoded, _ = self.encoder(torch.cat([emb, ctx], -1))

        pos_scores = []
        for i in range(L):
            is_blank = (word_state[:, i] == 0).float().unsqueeze(1)
            bc = blank_count.unsqueeze(1).float() / L
            pos_input = torch.cat([is_blank, bc, cnn_feat[:, i, :]], -1)
            pos_scores.append(self.pos_prior_mlp(pos_input).unsqueeze(1))
        priors = torch.cat(pos_scores, 1)  # [B, L, 26]

        out = torch.zeros(B, L, 26, device=word_state.device)
        for i in range(L):
            h = encoded[:, i, :]
            ptype = position_context[:, i]
            inp = torch.cat([h, priors[:, i, :]], -1)
            out[ptype==1, i, :] = self.left_decoder(inp[ptype==1])
            out[ptype==2, i, :] = self.right_decoder(inp[ptype==2])
            out[ptype==3, i, :] = self.both_decoder(inp[ptype==3])

        return out

class HangmanSolver:
    def __init__(self, word_list, model_path="best_model.pth"):
        self.model = EnhancedHangmanModel()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # Load state dict without DataParallel
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))

        # Now optionally wrap in DataParallel AFTER loading
        if torch.cuda.device_count() > 1:
            self.model = nn.DataParallel(self.model)

        self.model.eval()

        self.dictionary = word_list
        self.char_to_idx = {c: i+1 for i, c in enumerate(string.ascii_lowercase)}
        self.char_to_idx['_'] = 0
        self.idx_to_char = {v: k for k, v in self.char_to_idx.items() if v != 0}

    def _fallback_prediction(self, pattern, guessed):
        counter = Counter()
        for word in self.dictionary:
            if len(word) != len(pattern):
                continue
            match = True
            for wc, pc in zip(word, pattern):
                if pc != '_' and pc != wc:
                    match = False
                    break
                if pc == '_' and wc in guessed:
                    match = False
                    break
            if match:
                for i, c in enumerate(word):
                    if pattern[i] == '_' and c not in guessed:
                        counter[c] += 1

        if not counter:
            for c in string.ascii_lowercase:
                if c not in guessed:
                    return c
            return random.choice([c for c in string.ascii_lowercase if c not in guessed])  # final fallback

        for letter, _ in counter.most_common():
            if letter not in guessed:
                return letter

        return random.choice([c for c in string.ascii_lowercase if c not in guessed])

    def predict_letter(self, word_state, guessed_letters=None):
        if guessed_letters is None:
            guessed_letters = set()
        if ' ' in word_state:
            word_state = word_state.replace(' ', '')
        if all(c == '_' for c in word_state):
            # Return most frequent unguessed letter as the first guess
            common_order = "etaoinshrdlucmfwypvbgkjqxz"
            for letter in common_order:
                if letter not in guessed_letters:
                    return letter
        print("ok")
        word_state = word_state.lower()
        max_length = 45
        state_indices = []
        position_context = []

        for i, char in enumerate(word_state):
            if char == '_':
                state_indices.append(0)
                context = 0
                if i > 0 and word_state[i-1] != '_':
                    context += 1
                if i < len(word_state)-1 and word_state[i+1] != '_':
                    context += 2
                position_context.append(context)
            else:
                state_indices.append(self.char_to_idx.get(char, 27))
                position_context.append(0)

        while len(state_indices) < max_length:
            state_indices.append(27)
            position_context.append(0)

        word_tensor = torch.tensor([state_indices], dtype=torch.long).to(self.device)
        context_tensor = torch.tensor([position_context], dtype=torch.long).to(self.device)
        length_tensor = torch.tensor([len(word_state)], dtype=torch.long).to(self.device)
        blank_count_tensor = torch.tensor([word_state.count('_')], dtype=torch.long).to(self.device)
        blank_vowel_next = [0]*max_length

        for i in range(len(word_state)):
            if word_state[i] == '_':
                l = word_state[i-1] if i > 0 else 'x'
                r = word_state[i+1] if i < len(word_state)-1 else 'x'
                if l in 'aeiou' or r in 'aeiou':
                    blank_vowel_next[i] = 1

        blank_vowel_tensor = torch.tensor([blank_vowel_next], dtype=torch.float).to(self.device)

        with torch.no_grad():
            predictions = self.model(word_tensor, context_tensor, length_tensor, blank_count_tensor, blank_vowel_tensor)

        best_predictions = []
        for i in range(len(word_state)):
            if word_state[i] == '_' and position_context[i] > 0:
                probs = torch.softmax(predictions[0, i, :], dim=0)
                for j, prob in enumerate(probs):
                    letter = chr(ord('a') + j)
                    if letter not in guessed_letters:
                        best_predictions.append((letter, prob.item(), i))

        if not best_predictions:
            for i in range(len(word_state)):
                if word_state[i] == '_':
                    probs = torch.softmax(predictions[0, i, :], dim=0)
                    for j, prob in enumerate(probs):
                        letter = chr(ord('a') + j)
                        if letter not in guessed_letters:
                            best_predictions.append((letter, prob.item(), i))

        if best_predictions:
            best_predictions.sort(key=lambda x: x[1], reverse=True)
            return best_predictions[0][0]

        # Fallback to frequency-based guess if model fails
        common_order = "etaoinshrdlucmfwypvbgkjqxz"
        for letter in common_order:
            if letter not in guessed_letters:
                return letter

        return 'e'  # very rare fallback if all else fails

from tqdm import tqdm

def train_model(words, epochs=10, early_stopping_patience=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = EnhancedHangmanModel()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(opt, patience=2)
    loss_fn = nn.CrossEntropyLoss()

    best = float('inf')
    patience_counter = 0

    for ep in range(epochs):
        # Curriculum: reveal_ratio increases with epoch (starts hard, becomes easier)
        reveal_ratio = max(0.1, 0.55 - (ep) * 0.1)  # Start from 1.0, decrease to 0.1
        ds = HangmanDataset(words, reveal_ratio=reveal_ratio)
        train_len = int(0.9 * len(ds))
        tr, val = random_split(ds, [train_len, len(ds)-train_len])
        dl = DataLoader(tr, shuffle=True, pin_memory=True, batch_size=256, num_workers=2)
        vl = DataLoader(val, pin_memory=True, batch_size=256, num_workers=2)

        print(f"\n--- Epoch {ep+1} | Reveal Ratio: {reveal_ratio:.2f} ---", flush=True)
        model.train()
        total_loss = 0
        batch_count = 0

        for i, batch in enumerate(tqdm(dl, desc="Training", ncols=100)):
            opt.zero_grad()
            out = model(batch['word_state'].to(device), batch['position_context'].to(device),
                        batch['word_length'].to(device), batch['blank_count'].to(device),
                        batch['next_to_vowel'].to(device))
            loss, count = 0, 0
            for b in range(out.size(0)):
                target_pos = batch['target_positions'][b].to(device)
                target_char = batch['target_chars'][b].to(device)
                for p, c in zip(target_pos, target_char):
                    if p >= 0 and c > 0:
                        loss += loss_fn(out[b, p], c-1)
                        count += 1
            if count > 0:
                loss = loss / count
                loss.backward()
                opt.step()
                total_loss += loss.item()
                batch_count += 1
            if i % 20 == 0:
                if isinstance(loss, torch.Tensor):
                    print(f"  Batch {i}/{len(dl)} | Loss: {loss.item():.4f}", flush=True)
                else:
                    print(f"  Batch {i}/{len(dl)} | Loss: N/A (no valid targets)", flush=True)


        train_loss = total_loss / batch_count if batch_count > 0 else 0

        # Validation
        model.eval()
        val_loss = 0
        val_batches = 0
        with torch.no_grad():
            for batch in tqdm(vl, desc="Validation", ncols=100):
                out = model(batch['word_state'].to(device), batch['position_context'].to(device),
                            batch['word_length'].to(device), batch['blank_count'].to(device),
                            batch['next_to_vowel'].to(device))
                loss, count = 0, 0
                for b in range(out.size(0)):
                    target_pos = batch['target_positions'][b].to(device)
                    target_char = batch['target_chars'][b].to(device)
                    for p, c in zip(target_pos, target_char):
                        if p >= 0 and c > 0:
                            loss += loss_fn(out[b, p], c-1)
                            count += 1
                if count > 0:
                    val_loss += loss.item() / count
                    val_batches += 1

        val_loss = val_loss / val_batches if val_batches > 0 else 0
        scheduler.step(val_loss)

        if count > 0 and i % 20 == 0:
            print(f"  Batch {i}/{len(dl)} | Loss: {loss.item():.4f}", flush=True)

        if val_loss < best:
            best = val_loss
            patience_counter = 0
            torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), "best_model.pth")
            print("✅ Model improved and saved.", flush=True)
        else:
            patience_counter += 1
            print(f"⚠️ No improvement. Patience: {patience_counter}/{early_stopping_patience}", flush=True)
            if patience_counter >= early_stopping_patience:
                print(f"🛑 Early stopping at epoch {ep+1}", flush=True)
                break

    return model

In [ ]:
dictionary = open("/content/words_250000_train.txt").read().splitlines()
# Test code to run in a separate cell after defining the model classes
random.shuffle(dictionary)
word_list = dictionary[:12000]

print("Starting training...")
print(f"Training on {len(word_list)} words")
model = train_model(word_list, epochs=10)

print("\nTraining completed!")
torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), "best_model.pth")
print("Model saved as 'best_model.pth'")

solver = HangmanSolver(word_list)
solver.model = model
solver.model.eval()

# Game simulation
def simulate_hangman_game(word, solver, max_wrong=6, verbose=True):
    word = word.lower()
    current = '_' * len(word)
    guessed = set()
    wrong_count = 0
    moves = []

    if verbose:
        print(f"\n{'='*30}\nGAME: {word}\n{'='*30}")

    while '_' in current and wrong_count < max_wrong:
        if verbose:
            print(f"Current: {current}\nWrong guesses: {wrong_count}/{max_wrong}")

        letter = solver.predict_letter(current, guessed)
        guessed.add(letter)
        moves.append(letter)

        if letter in word:
            current = ''.join([c if c == letter or current[i] != '_' else '_' for i, c in enumerate(word)])
            if verbose:
                print(f"✓ Correct! Guessed: {letter}")
        else:
            wrong_count += 1
            if verbose:
                print(f"✗ Wrong! Guessed: {letter}")
        if verbose:
            print()

    if verbose:
        if '_' not in current:
            print(f"🎉 WON! Final word: {current}")
        else:
            print(f"💀 LOST! Word was: {word}")
        print(f"Moves: {' -> '.join(moves)}")

    return '_' not in current, moves, wrong_count

# Run game test
simulate_hangman_game("learning", solver, verbose=True)

Starting training...
Training on 12000 words

--- Epoch 1 | Reveal Ratio: 0.55 ---


Training:   0%|                                                            | 0/3375 [00:00<?, ?it/s]

  Batch 0/3375 | Loss: 3.2584


Training:   1%|▎                                                  | 20/3375 [00:16<49:47,  1.12it/s]

  Batch 20/3375 | Loss: 2.8049


...

Training:  59%|████████████████████████████▋                    | 1980/3375 [20:58<15:27,  1.50it/s]

  Batch 1980/3375 | Loss: 2.1600


Training:  59%|█████████████████████████████                    | 2000/3375 [21:11<14:39,  1.56it/s]

  Batch 2000/3375 | Loss: 2.3219


Training:  59%|█████████████████████████████                    | 2002/3375 [21:12<14:15,  1.60it/s]